In [2]:
import mediapipe as mp
import cv2
import numpy as np
import os

In [11]:
def extract_keypoints(results):
    '''
    Extract Face Keypoints
    '''
    face_keypoints = np.array([[res.x, res.y, res.z] for res in results.multi_face_landmarks[0].landmark]).flatten() if results.multi_face_landmarks else np.zeros(468*3)
    return face_keypoints

In [15]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
max_num_faces=1,
min_detection_confidence=0.3,
min_tracking_confidence=0.8
)

mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

In [13]:
cap = cv2.VideoCapture(0)

counter = 0
drowsiness = ""
status = ""

while cap.isOpened():
    ret, frame = cap.read()
    image = cv2.flip(frame, 1)
    img_h, img_w, channels = frame.shape
    
    rgb_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_h, img_w = frame.shape[:2]
    rgb_frame.flags.writeable = False
    results = face_mesh.process(rgb_frame)
    rgb_frame.flags.writeable = True
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list = face_landmarks,
                    connections = mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec = drawing_spec,
                    connection_drawing_spec = drawing_spec)
    
    
    cv2.imshow("Feed", image)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [6]:
cap.release()
cv2.destroyAllWindows()

In [14]:
extract_keypoints(results)

array([ 0.45648155,  0.66380548, -0.03342569, ...,  0.5769732 ,
        0.47035497,  0.00589202])

In [49]:
!rm -rf "Drowsiness Detection"
!mkdir "Drowsiness Detection"

In [50]:
import os

In [62]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join(os.getcwd() ,'Drowsiness Detection') 

# Actions that we try to detect
# actions = np.array(['neutral', 'drowsy', 'sleep'])
actions = np.array(['drowsy'])

# Thirty videos worth of data
no_sequences = 350

# Videos are going to be 30 frames in length
sequence_length = 5

In [63]:
actions

array(['drowsy'], dtype='<U6')

In [59]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [ ]:
stream = cv2.VideoCapture(0)

for action in actions:
    for sequence in range(no_sequences):
        for frame_num in range(sequence_length):
            ret, frame = stream.read()
            image = cv2.flip(frame, 1)

            rgb_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            rgb_frame.flags.writeable = False
            results = face_mesh.process(rgb_frame)
            rgb_frame.flags.writeable = True

            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    mp_drawing.draw_landmarks(
                            image=image,
                            landmark_list = face_landmarks,
                            connections = mp_face_mesh.FACEMESH_TESSELATION,
                            landmark_drawing_spec = drawing_spec,
                            connection_drawing_spec = drawing_spec)
            
            if frame_num == 0:
                cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('Data Collection', image)
            else:
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('Data Collection', image)
                
            # NEW Export keypoints
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
            np.save(npy_path, keypoints)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    stream.release()
    cv2.destroyAllWindows()

In [42]:
stream.release()
cv2.destroyAllWindows()